In [ ]:
import streamlit as st
import pandas as pd

st.set_page_config(page_title="삼성(블라인드) 데이터 대시보드", layout="wide")

CSV_PATH = r"C:\Users\USER\sesac-miniProject\완료\블라인드_삼성.csv"
 # 파일이 다른 위치면 경로만 수정하세요.

@st.cache_data(show_spinner=False)
def load_data(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)

    # 필수 컬럼 점검 (없으면 즉시 에러로 원인 파악)
    required = {"title", "date", "views", "likes", "comments", "content", "url"}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"CSV에 필수 컬럼이 없습니다: {sorted(missing)}")

    # 타입/결측 정리
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    for c in ["views", "likes", "comments"]:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).astype(int)

    for c in ["title", "content", "url"]:
        df[c] = df[c].fillna("").astype(str)

    # 최신순 정렬
    df = df.sort_values("date", ascending=False, na_position="last").reset_index(drop=True)
    return df


def apply_filters(df: pd.DataFrame) -> pd.DataFrame:
    st.sidebar.header("필터")

    # 날짜 범위(유효 날짜만)
    dmin = df["date"].min()
    dmax = df["date"].max()

    if pd.isna(dmin) or pd.isna(dmax):
        st.sidebar.warning("date 컬럼이 날짜로 파싱되지 않았습니다. (형식을 확인하세요)")
        start_date, end_date = None, None
    else:
        start_date, end_date = st.sidebar.date_input(
            "기간",
            value=(dmin.date(), dmax.date()),
            min_value=dmin.date(),
            max_value=dmax.date(),
        )

    keyword = st.sidebar.text_input("키워드(제목/본문)", value="").strip()
    min_views = st.sidebar.number_input("최소 조회수", min_value=0, value=0, step=10)
    min_likes = st.sidebar.number_input("최소 좋아요", min_value=0, value=0, step=1)
    min_comments = st.sidebar.number_input("최소 댓글수", min_value=0, value=0, step=1)

    out = df.copy()

    if start_date and end_date:
        out = out[(out["date"].dt.date >= start_date) & (out["date"].dt.date <= end_date)]

    if keyword:
        k = keyword.lower()
        out = out[
            out["title"].str.lower().str.contains(k, na=False)
            | out["content"].str.lower().str.contains(k, na=False)
        ]

    out = out[
        (out["views"] >= min_views)
        & (out["likes"] >= min_likes)
        & (out["comments"] >= min_comments)
    ]

    return out


def daily_agg(df: pd.DataFrame) -> pd.DataFrame:
    # 날짜(일 단위) 집계
    tmp = df.dropna(subset=["date"]).copy()
    if tmp.empty:
        return pd.DataFrame(columns=["day", "posts", "views", "likes", "comments"])

    tmp["day"] = tmp["date"].dt.date
    g = (
        tmp.groupby("day", as_index=False)
           .agg(
               posts=("title", "count"),
               views=("views", "sum"),
               likes=("likes", "sum"),
               comments=("comments", "sum"),
           )
           .sort_values("day")
           .reset_index(drop=True)
    )
    return g


# -------------------------
# 메인
# -------------------------
st.title("삼성(블라인드) 게시글 분석 대시보드")

df = load_data(CSV_PATH)
filtered = apply_filters(df)

# KPI
c1, c2, c3, c4 = st.columns(4)
c1.metric("게시글 수", f"{len(filtered):,}")
c2.metric("총 조회수", f"{filtered['views'].sum():,}")
c3.metric("총 좋아요", f"{filtered['likes'].sum():,}")
c4.metric("총 댓글수", f"{filtered['comments'].sum():,}")

st.divider()

# 일별 추이
st.subheader("일별 추이")
daily = daily_agg(filtered)

if daily.empty:
    st.info("선택한 조건에서 날짜 데이터가 없어서 추이 차트를 표시할 수 없습니다.")
else:
    chart_col1, chart_col2 = st.columns(2)

    with chart_col1:
        st.caption("게시글 수 / 조회수")
        tmp = daily.set_index("day")[["posts", "views"]]
        st.line_chart(tmp)

    with chart_col2:
        st.caption("좋아요 / 댓글수")
        tmp = daily.set_index("day")[["likes", "comments"]]
        st.line_chart(tmp)

st.divider()

# Top 게시글
st.subheader("Top 게시글")
sort_key = st.selectbox("정렬 기준", ["views", "likes", "comments", "date"], index=0)
ascending = st.checkbox("오름차순", value=False)

view_cols = ["date", "title", "views", "likes", "comments", "url"]
topn = st.slider("표시 개수", min_value=10, max_value=200, value=50, step=10)

table = filtered.sort_values(sort_key, ascending=ascending, na_position="last")[view_cols].head(topn)

# URL 클릭 가능하게(표는 그대로, 아래에 링크 리스트 제공)
st.dataframe(table, use_container_width=True)

with st.expander("표에 있는 URL 링크 열기"):
    for _, r in table.iterrows():
        if r["url"]:
            st.markdown(f"- [{r['title'][:80]}]({r['url']})")

st.divider()

# 다운로드
st.subheader("필터 결과 다운로드")
csv_bytes = filtered.to_csv(index=False, encoding="utf-8-sig").encode("utf-8-sig")
st.download_button(
    label="CSV로 다운로드",
    data=csv_bytes,
    file_name="blind_samsung_filtered.csv",
    mime="text/csv",
)


2026-01-21 11:07:53.267 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-21 11:07:53.271 No runtime found, using MemoryCacheStorageManager
2026-01-21 11:07:53.279 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-21 11:07:53.281 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-21 11:07:53.283 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


2026-01-21 11:07:53.435 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-21 11:07:53.437 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-21 11:07:53.438 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-21 11:07:53.445 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-21 11:07:53.447 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-21 11:07:53.449 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-21 11:07:53.456 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-21 11:07:53.463 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

False